## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [22]:
%%sql
postgresql:///animal_shelters

WITH tb_animals AS(
    SELECT animalid,
        animaltype,
        CAST(('12/31/2021'::date - birthdate::date)/365 AS TEXT) AS age,
        CASE WHEN animaltype = 'Dog' AND weight <= 10.0 then 'DS'
           WHEN animaltype = 'Dog' AND weight > 10.0 AND weight <= 30.0 then 'DM'
           WHEN animaltype = 'Dog' AND weight >30.0 then 'DL'
           WHEN animaltype = 'Cat' AND weight <= 5.0 then 'CS'
           WHEN animaltype = 'Cat' AND weight > 5.0 AND weight <= 7.0 then 'CM'
           WHEN animaltype = 'Cat' AND weight > 7.0 then 'CL'
           WHEN animaltype = 'Bird' AND weight <= 0.7 then 'BS'
           WHEN animaltype = 'Bird' AND weight > 0.7 AND weight <= 1.1 then 'BM'
           WHEN animaltype = 'Bird' AND weight > 1.1 then 'BL'
             END AS sizeid,
        location
    FROM animals),

    tb_joins AS(
    SELECT ani.animalid,
           loc.costs AS location_costs,
           age.costs AS age_costs,
           size.costs AS size_costs,
           ani.animaltype AS animaltype,
           size.size AS size
    FROM tb_animals AS ani
        LEFT JOIN location_costs AS loc ON ani.location = loc.location
        LEFT JOIN age_costs AS age ON ani.age = age.age
        LEFT JOIN size_costs AS size ON ani.sizeid = size.sizeid
        LEFT JOIN sponsored_animals AS spa ON ani.animalid = spa.sponsorid
    WHERE spa.sponsorid IS NULL)

--SELECT * FROM tb_joins LIMIT 10

SELECT LOWER(animaltype) AS animaltype, LOWER(size) AS size,
       SUM(location_costs + age_costs + size_costs) AS total,
       ROUND(SUM(location_costs + age_costs + size_costs) * 100.0 / SUM(SUM(location_costs + age_costs + size_costs)) OVER (),2) AS percentage
       FROM tb_joins
       GROUP BY animaltype,size
       ORDER BY animaltype, size DESC

9 rows affected.


animaltype,size,total,percentage
bird,small,1615,0.05
bird,medium,3460,0.10
bird,large,7770,0.22
cat,small,518015,14.89
cat,medium,250575,7.20
cat,large,439490,12.64
dog,small,336680,9.68
dog,medium,942095,27.09
dog,large,978115,28.12


In [23]:
%%sql
postgresql:///animal_shelters

WITH data as (
  select 
    lower(a.animaltype) as animaltype, 
    lower(sc.size) as size, 
    sum(ac.costs + lc.costs + sc.costs) as cost 
  from 
    animals a 
    left join age_costs as ac on ac.age = (
      date_part('year', '2021-12-31' :: date) - date_part('year', a.birthdate :: date)
    ):: varchar 
    left join location_costs lc on lc.location = a.location 
    left join sponsored_animals sa on sa.sponsorid = animalid 
    left join size_costs sc on sc.size = case when a.animaltype = 'Dog' then (
      case when weight <= 10 then 'Small' when weight <= 30 then 'Medium' else 'Large' end
    ) when a.animaltype = 'Cat' then (
      case when weight <= 5 then 'Small' when weight <= 7 then 'Medium' else 'Large' end
    ) when a.animaltype = 'Bird' then (
      case when weight <= 0.7 then 'Small' when weight <= 1.1 then 'Medium' else 'Large' end
    ) end 
    and sc.animaltype = a.animaltype 
  where 
    sa.sponsorid is null 
  group by 
    1, 
    2
) 
select 
  animaltype, 
  size, 
  cost as total, 
  round(
    (0.0 + cost)* 100 / sum(cost) OVER (), 
    2
  ) as percentage 
from 
  data 
order by 
  animaltype, 
  size desc

-- resposta correta; a diferença é minima em relação a tabela anterior
-- diferindo apenas na tabela dos cachorros (150,200 e 450 a mais para cada categoria, respectivamente)

9 rows affected.


animaltype,size,total,percentage
bird,small,1615,0.05
bird,medium,3460,0.10
bird,large,7770,0.22
cat,small,518015,14.90
cat,medium,250575,7.21
cat,large,439490,12.64
dog,small,336530,9.68
dog,medium,941895,27.09
dog,large,977665,28.12
